In [1]:
import pandas as pd
df=pd.read_csv(r"CSV/train.csv")

In [2]:
z = (df['Annual Income']-df['Annual Income'].mean())/df['Annual Income'].std()
df['z score'] = z
df = df[(df['z score']<=3) & (df['z score']>=-3)]

df['Marital Status'] = df[['Marital Status']].fillna(df['Marital Status'].mode()[0])
df['Age'] = df[['Age']].fillna(df['Age'].median())
df['Number of Dependents'] = df[['Number of Dependents']].fillna(df['Number of Dependents'].median())
df['Previous Claims'] = df[['Previous Claims']].fillna(df['Previous Claims'].median())
df['Occupation'] = df[['Occupation']].fillna(df['Occupation'].mode()[0])
df['Health Score'] = df[['Health Score']].fillna(df['Health Score'].median())
df['Credit Score'] = df[['Credit Score']].fillna(df['Credit Score'].median())
df['Customer Feedback'] = df[['Customer Feedback']].fillna(df['Customer Feedback'].mode()[0])
df['Vehicle Age'] = df[['Vehicle Age']].fillna(df['Vehicle Age'].median())
df['Insurance Duration'] = df[['Insurance Duration']].fillna(df['Insurance Duration'].median())


In [3]:
df['Gender']=df['Gender'].replace({'Female':0,'Male':1})
df['Smoking Status']=df['Smoking Status'].replace({'No':0,'Yes':1})
df=pd.get_dummies(df,columns=['Property Type',
'Marital Status',
'Education Level',
'Occupation',
'Location',
'Policy Type',
'Customer Feedback',
'Exercise Frequency'],
dtype='int')
df.drop(['Policy Start Date'],axis=1,inplace=True)

C:\Users\shree\AppData\Local\Temp\ipykernel_19832\1876511553.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender']=df['Gender'].replace({'Female':0,'Male':1})
C:\Users\shree\AppData\Local\Temp\ipykernel_19832\1876511553.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Smoking Status']=df['Smoking Status'].replace({'No':0,'Yes':1})


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
X=df.drop(['Premium Amount'],axis=1)
# X.isna().sum()
y=df['Premium Amount']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [5]:
# params={
#     "n_estimators":25,
#     "max_depth":10,
#     "n_jobs":-1,
# }
# import mlflow
# mlflow.set_experiment('First exp')
# mlflow.set_tracking_uri("http://127.0.0.1:5000")

# with mlflow.start_run():
#     mlflow.log_params(params)
#     mlflow.log_metric('r2',r2_score(y_test,test_pred))
#     mlflow.sklearn.log_model(model,"Random Forest Regressor")

In [6]:
import xgboost as xgb
models=[(
    "RandomForestRegressor",
    {"n_estimators":40,"max_depth":5,"n_jobs":-1},
    RandomForestRegressor(),
    (x_train,y_train),
    (x_test,y_test)
    
),
(
    "XGBRegressor",
    {'objective':'reg:squarederror',"n_estimators":40,"max_depth":5,"n_jobs":-1},
    xgb.XGBRegressor(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "LinearRegression",
    {},
    LinearRegression(),
    (x_train,y_train),
    (x_test,y_test)

),
(
    "DecisionTreeRegressor",
    {},
    DecisionTreeRegressor(),
    (x_train,y_train),
    (x_test,y_test)

)
]

In [20]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
reports=[]
for model_name,params,model,train_set,test_set in models:
    x_train=train_set[0]
    y_train=train_set[1]
    x_test=test_set[0]
    y_test=test_set[1]

    model.set_params(**params)
    model.fit(x_train,y_train)
    train_pred = model.predict(x_train)  # Added this line
    y_pred=model.predict(x_test)

    rmse=np.sqrt(mean_squared_error(y_test, y_pred))
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_train,train_pred)
    rsmle = np.sqrt(mean_squared_log_error(y_train, np.maximum(train_pred, 0)))
    reports.append((model_name,rmse,mae,r2,rsmle))

In [21]:
import mlflow

mlflow.set_experiment('Smart_Premium_MLs')
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metric('RMSE', report[1])  # ✅ Fixed syntax
        mlflow.log_metric('MAE', report[2])
        mlflow.log_metric('R2', report[3])
        mlflow.log_metric('RSMLE', report[4])
        
        if 'XGBRegressor' in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/03/31 10:25:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
D:\conda\Lib\site-packages\xgboost\core.py:158: UserWarning: [10:25:44] WARNING: D:\bld\xgboost-split_1737531311373\work\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


🏃 View run RandomForestRegressor at: http://127.0.0.1:5000/#/experiments/267407948755392247/runs/81813e1ca41b467582fd5160322fee11
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267407948755392247


2025/03/31 10:25:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBRegressor at: http://127.0.0.1:5000/#/experiments/267407948755392247/runs/5348ca4c559a4b7ba2302e3fc12e9f80
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267407948755392247


2025/03/31 10:25:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LinearRegression at: http://127.0.0.1:5000/#/experiments/267407948755392247/runs/f3ad0419f48c4a6189856981c21767a8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267407948755392247


2025/03/31 10:25:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTreeRegressor at: http://127.0.0.1:5000/#/experiments/267407948755392247/runs/0bda8729c1b54104a0e1b1a5e0435541
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267407948755392247
